# **Organización de Datos 75.06 - Trabajo Práctico N°1**
N6. Queremos predecir el género de la película según su descripción. Para esto vamos a buscar para una descripción nueva los 10 vecinos más cercanos y asignar el género que más se repita entre ellos. Pruebe este algoritmo para 5 películas al azar y para una descripción inventada que se le ocurra para una película nueva de Disney (su corrector valorará si logra hacerlo reír (? ).

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')

peliculas = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/csv/movies.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df_limpio = peliculas[ ['title','genres','overview'] ].dropna().reset_index()
overview_mas_largo = df_limpio.overview.map(lambda x: len(x)).max()

stopwordsEng = set(stopwords.words('english'))
countVec = CountVectorizer(lowercase = True, stop_words = stopwordsEng, max_features = overview_mas_largo)
buscaVecinos = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
matrizVectores = countVec.fit_transform(df_limpio.overview)
buscaVecinos.fit(matrizVectores)

def vecinos_pelicula(pelicula):
  busqueda = countVec.transform(pelicula.overview)
  distancias, indices = buscaVecinos.kneighbors(busqueda, n_neighbors = 10)

  df_vecinos = df_limpio.loc[df_limpio.index[indices[0]]]
  df_vecinos.genres = df_vecinos.genres.map(lambda x: x.split(','))
  df_vecinos = df_vecinos.explode('genres')

  generos = df_vecinos.genres.mode().tolist()
  print("\nLos géneros de", pelicula.title.values[0], "son:", pelicula.genres.values[0])
  print("Los generos mas recurrentes de las 10 pelis vecinas son:", generos)

for i in range(5):
  vecinos_pelicula(df_limpio.sample()) #Obtenemos una peli random.

rey_leon = {'title': ['The Lion King'], 'genres': ['Animation,Drama,Musical'], 'overview': ["Prince's uncle kills father so he enlists the help of two immature vagrants to exact revenge. He falls in love and marries his cousin after he kills his uncle, taking his place as king."]}
rey_leon_df = pd.DataFrame(data=rey_leon)
vecinos_pelicula(rey_leon_df)


Los géneros de Recipe for a Perfect Christmas son: Comedy
Los generos mas recurrentes de las 10 pelis vecinas son: ['Drama']

Los géneros de Assassin of the Tsar son: History,Drama
Los generos mas recurrentes de las 10 pelis vecinas son: ['Drama']

Los géneros de Heights son: Drama,Romance
Los generos mas recurrentes de las 10 pelis vecinas son: ['Drama']

Los géneros de The Nightmare Before Christmas son: Fantasy,Animation,Family
Los generos mas recurrentes de las 10 pelis vecinas son: ['Family']

Los géneros de Harriet Craig son: Drama
Los generos mas recurrentes de las 10 pelis vecinas son: ['Comedy', 'Drama']

Los géneros de The Lion King son: Animation,Drama,Musical
Los generos mas recurrentes de las 10 pelis vecinas son: ['Drama']
